In [1]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import json
from networkx.readwrite import json_graph


In [2]:
import findspark
findspark.find()
findspark.init(edit_profile=True)
import pyspark
sc = pyspark.SparkContext()

In [7]:
with open("../../graph/US.json", "r") as graph_data:
    us_data = json.load(graph_data)
    us_digraph = json_graph.node_link_graph(us_data)
    

In [9]:
us_empty_nodes = [i for i in us_digraph.nodes() if not us_digraph.succ[i]]

In [10]:
us_N_neighbor_nodes = [i for i in us_digraph.nodes() if len(us_digraph.succ[i]) >= 500]

In [11]:
len(us_N_neighbor_nodes)

197

In [17]:
start = time.time()
influence_list = {}
for i in us_N_neighbor_nodes:
    influence_list[i] = cascade([i], us_digraph)
    print i, ':', influence_list[i]
sorted_max_potential_nodes = sorted(influence_list.items(), key=lambda x: x[1], reverse=True )

print 'max potential node set', sorted_max_potential_nodes[:3]

Kqvfep2mxS10S50FbVDi4Q : 183899
kpbhy1zPewGDmdNfNqQp-g : 183950
ChgmBht69bnqSdDxpWMAqQ : 184189
6l_ATA06SBvvPDMvMiKQgw : 183784
lRfdcptGBxolpkBPlZ358A : 184078
4LOUTfHA1tZQ7qyNDLQuow : 184164
3gIfcQq5KxAegwCPXc83cQ : 184209
xT3nCjsC6LHtrid-0LWQ_w : 184100
twtWSkxomLIpDplupkVoVw : 184173
WmAyExqSWoiYZ5XEqpk_Uw : 184165
auVkTXPmA8o3oU8barlBSw : 184057
xV1hf20ZFDSoaMZmwLoHNg : 184020
k5p3YP1ZjCa8ZS3xqXgBug : 183902
zTWH9b_ItSdLOK9ypeFOIw : 183915
kGgAARL2UmvCcTRfiscjug : 184387
i8KMQX7nekLSCzb_DVb6yA : 183912
gzJpPaHN-NXBkAZcZri3hw : 184197
uZbTb-u-GVjTa2gtQfry5g : 184029
9A2-wSoBUxlMd3LwmlGrrQ : 184005
Oa2GOM0oMhVxbfh-53N88w : 184098
ri0xJJElIs47Tm1xvn1dOA : 183962
knZduWPnQA0kQ9Rrq4lUhg : 183951


KeyboardInterrupt: 

In [18]:
def cascade(init_nodes, nodes_set_broadcast):#, dist_d):
    nodes_set = nodes_set_broadcast
    action = {}
    n = len(init_nodes)
    #np.random.seed(random_d)
    #init_nodes = np.random.choice(NC_digraph.nodes(), 1)[0]
    for i in init_nodes:
        action[i] = 1
    #st = set()
    #st.add(init_nodes)
    init_list = zip([0]*len(init_nodes),init_nodes[::])
    inter = 0
    while len(init_list) != 0:
        curr_node = init_list.pop(0)
        #print curr_node
        for i in nodes_set[curr_node[1]]:
            if i not in action:
                b = nodes_set.node[i]['review_count']
                a =  nodes_set[curr_node[1]][i]['weight']
                #np.random.seed(12)
                b_dist = np.sqrt(np.random.beta(a = a, b = b))
                #np.random.seed(12)
                u_dist = np.random.uniform(0,1)
                if b_dist > u_dist:
                    action[i] = 1
                    #st.add(i)
                    inter = curr_node[0] + 1
                    init_list.append((inter, i))
                    n = n + 1
    return n

def influence_function(N, init_nodes, partition_num):
    nodes_set_broadcast = sc.broadcast(nc_full_digraph)
    activated_num_rdd = sc.parallelize([init_nodes]*N, partition_num)
    activated_num = activated_num_rdd.map(lambda x: cascade(x, nodes_set_broadcast)).reduce(lambda x, y: x+y)
    return activated_num/N
####################################################################################


######################################################################################
#
#Simulated Annealing Algorithm Implementation
#
######################################################################################

def next_step(X, nodes_set):
    tmp = X[0]
    while tmp in X:
        tmp = np.random.choice(nodes_set)
    X[np.random.choice(range(len(X)))] = tmp
    return X


def simulated_annealing_rst(function, N, partition_num, initial_X, initial_temp, cool, reanneal, iterr, nodes_set):
    
    accepted = 0
    #X = initial_X.copy()
    X = initial_X[::]
    T = initial_temp
    
    history = list()
    # Evaluate E
    prev_E = function(N, X, partition_num)
    history.append(prev_E)
    
    for i in xrange(iterr):
        # Propose new path.
        X_star = next_step(X, nodes_set)
        # Evaluate E
        new_E = function(N, X_star, partition_num)
        delta_E = new_E - prev_E
        
        # Flip a coin
        U = np.random.uniform()
        
        if U < np.exp(delta_E / T):
            accepted += 1
            history.append(new_E)
            print 'value:', new_E
            # Copy X_star to X
            print "this is", i, "iteration"
            #X = X_star.copy()
            X = X_star[::]
            print 'set:', X
            prev_E = new_E
        
        # Check to cool down
        if accepted % reanneal == 0:
            T *= cool
    return X, history